<a href="https://colab.research.google.com/github/wendyamarteifio/AZUBI-/blob/master/AzubiML_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
 #Importing the pandas library as a variable names np
 import pandas as pd
#Importing numpy library as a variable named np
 import numpy as np
 #Importing matplotlib.pyplot as a varaible named plt
 import matplotlib.pyplot as plt
#Importing seaborn as a variable named sns 
 import seaborn as sns
#this is the magical function 
 %matplotlib inline

In [36]:
#Importing sklearn.datasets package to practice on some toy datasets 
from sklearn.datasets import load_boston
#Setting the variable boston to boston dataset
boston=load_boston()

In [ ]:
#Gives a detailed information on the boston data
print(boston.DESCR)

In [ ]:
# Displays column names of dataset
boston.feature_names

In [ ]:
#Contains the dataset within the boston variable
boston

In [40]:
#Creating a dataframe out of the boston dataset and column names 
df=pd.DataFrame(boston.data, columns=boston.feature_names)


In [ ]:
# displays summary of data.
df.head()

In [42]:
#Calling house prices with .target and storing it in the dataframe
df['MEDV']=boston.target

In [ ]:
#displaying the dataset
df.head()

In [ ]:
#Gives a concise summary of dataframe 
df.info()

In [ ]:
#Computes the summary statistics for the dataframe columns
df.describe()

In [ ]:
#Plots a pairwise relationship in the dataset 
sns.pairplot(df)

In [ ]:
#seaborn.distplot will allow you easily fit a normal distrubution to each histogram plot and make visualisations easier 
rows=7
cols=2
fig, ax=plt.subplots (nrows=rows, ncols=cols, figsize=(16,16))
col=df.columns
index=0

for i in range(rows):
  for j in range(cols):
    sns.distplot(df[col[index]], ax=ax[i][j])
    index=index+1
plt.tight_layout()


In [ ]:
#
fig, ax=plt.subplots(figsize=(16,9))
sns.heatmap(df.corr(), annot=True, annot_kws={'size':12})

In [48]:
#For a linear regression model, we need high correlations. in that case, we need to define a threshold filter
#To obtain a lindear regression 
def getCorrelatedFeature(corrdata, threshold):
  feature=[]
  value=[]

  for i, index in enumerate(corrdata.index):
    if abs(corrdata[index])>threshold:
      feature.append(index)
      value.append(corrdata[index])

  df=pd.DataFrame(data=value, index=feature, columns=['Corr Value'])
  return df


Checking the correlations of and summarizing the relationships between the variables. A correlation factor near 1 or -1 is wanted. 0 means that there is no linearrelation between that columns and they will sabotage the linear regression model

In [49]:
#Looking for magnitudes of correlation data that has magnitude greater than 0.4
threshold=0.4
#appending and updating the index column in corr_value
corr_value=getCorrelatedFeature(df.corr()['MEDV'], threshold)

In [ ]:
#Gets the actual value at a specified index 
corr_value.index.values


In [ ]:
#These outputs affect the proces according to the threshold given 
#Storing the indexes of the correlation values
correlated_data=df[corr_value.index]
#Display dataframe correlated_data 
correlated_data.head()

Train/Test Split involves splitting the dataset into training and testing sets respectively, which are mutually exclusive. After which, you train with the training set and test with the testing set. This will provide a more accurate evaluation on out-of-sample accuracy because the testing dataset is not part of the dataset that have been used to train the data. It is more realistic for real world problems.

This means that we know the outcome of each data point in this dataset, making it great to test with! And since this data has not been used to train the model, the model has no knowledge of the outcome of these data points. So, in essence, it’s truly an out-of-sample testing.

In [52]:
X=correlated_data.drop(labels=['MEDV'], axis=1)
y=correlated_data['MEDV']

In [53]:
from sklearn.model_selection import train_test_split
#This means we are training the model with 67% of the dataset and the test model represents 33%
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.33,random_state=1) 


In [54]:
from sklearn.linear_model import LinearRegression
lm=LinearRegression()

In [ ]:
lm.fit(X_train,y_train)

In [ ]:
predictions=lm.predict(X_test)
plt.scatter(y_test,predictions)

In [ ]:
sns.distplot((y_test-predictions),bins=50)

In [ ]:
lm.intercept_


In [ ]:
lm.coef_

Coefficient and Intercept are the parameters of the fit line. Given that it is a multiple linear regression and knowing that the parameters are the intercept and coefficients of hyperplane, sklearn can estimate them from our data. Scikit-learn uses plain Ordinary Least Squares method to solve this problem.

Ordinary Least Squares (OLS)
OLS is a method for estimating the unknown parameters in a linear regression model. OLS chooses the parameters of a linear function of a set of explanatory variables by minimizing the sum of the squares of the differences between the target dependent variable and those predicted by the linear function. In other words, it tries to minimizes the sum of squared errors (SSE) or mean squared error (MSE) between the target variable (y) and our predicted output ( ℎ𝑎𝑡𝑦 ) over all samples in the dataset.

OLS can find the best parameters using of the following methods:

In [60]:
def lin_func(values, coefficients=lm.coef_, y_axis=lm.intercept_):
  return np.dot(values, coefficients)+y_axis

In [ ]:
from random import randint
for i in range(5):
  index=randint(0,len(df)-1)
  sample=df.iloc[index][corr_value.index.values].drop('MEDV')

  print('PREDICTION:',round(lin_func(sample),2),
        '// REAL:',df.iloc[index]['MEDV'],
        '// DIFFERENCE:',round(round(lin_func(sample),2)-df.iloc[index]['MEDV'],2)
        )